<a href="https://colab.research.google.com/github/djdongjin/IFT6135-Assignment/blob/master/A1_2_Jin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as pltt
import os

torch.manual_seed(10)
torch.cuda.manual_seed(10)
np.random.seed(10)

In [77]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
batch_size = 200
lr = 1e-2
epochs = 10

In [0]:
def one_hot(labels, n):
    """labels: m*1 vector
       n: expected classes
       outout: m*n matrix"""
    m = len(labels)
    onehot = np.zeros((m, n))
    onehot[np.arange(m), labels] = 1
    return onehot

DATA_PATH = r'/content/gdrive/My Drive/Datasets/MNIST'
X_train = torch.from_numpy(np.load(DATA_PATH + '/x_train.npy')).reshape(-1,1,28,28)
y_train = torch.from_numpy(np.load(DATA_PATH + '/y_train.npy')).type(torch.LongTensor)
X_val   = torch.from_numpy(np.load(DATA_PATH + '/x_val.npy')).reshape(-1,1,28,28)
y_val   = torch.from_numpy(np.load(DATA_PATH + '/y_val.npy')).type(torch.LongTensor)
X_test  = torch.from_numpy(np.load(DATA_PATH + '/x_test.npy')).reshape(-1,1,28,28)
y_test  = torch.from_numpy(np.load(DATA_PATH + '/y_test.npy')).type(torch.LongTensor)

train_set = TensorDataset(X_train, y_train)
val_set   = TensorDataset(X_val, y_val)
test_set  = TensorDataset(X_test, y_test)

dev = torch.device('cpu') if not torch.cuda.is_available() else torch.device('cuda:0')

In [0]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        
        self.criterion = nn.CrossEntropyLoss()
        
        self.conv1 = nn.Conv2d(1, 48, kernel_size=(11,11), stride=1) # 18
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=2, padding=1) # 10
        self.conv2 = nn.Conv2d(48, 96, kernel_size=(1,1)) # 10
        self.conv3 = nn.Conv2d(96,256, kernel_size=(2,2)) # 5
        
        self.fc    = nn.Linear(256*5*5, 10)
        
        
    def forward(self, X):
        output = self.pool1(F.relu(self.conv1(X)))
        output = F.relu(self.conv2(output))
        output = self.pool1(F.relu(self.conv3(output)))
        output = output.view(-1, 256*5*5)
        output = self.fc(output)
        
        return output
        

In [0]:
def train(model, train_set, test_set, epochs, batch_size, optimizer):    
    global dev
    
    train_err, train_acc, val_err, val_acc = [], [], [], [] 
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    
    model.to(dev)
    
    for ep in range(1, epochs+1):
        err = 0.0
        acc = 0.0
        total = 0
        for i, (X,y) in enumerate(train_loader):
            X = X.to(dev)
            y = y.to(dev)
            
            optimizer.zero_grad()
            y_hat = model(X)
            loss = model.criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            total+= y.size(0)
            err += loss
            acc += torch.sum(torch.max(y_hat,1)[1] == y)
            
        train_err.append(err/total)
        train_acc.append(acc/total)
        
        err = 0.0; acc = 0.0; total = 0
        for i, (X,y) in enumerate(test_loader):
            X = X.to(dev)
            y = y.to(dev)
            with torch.no_grad():
                
                y_hat = model(X)
                loss  = model.criterion(y_hat, y)
                
                total += y.size(0)
                err += loss
                acc += torch.sum(torch.max(y_hat,1)[1] == y)
                
        val_err.append(err/total)
        val_acc.append(acc/total)
        
    return train_err, train_acc, val_err, val_acc        

In [0]:
model = AlexNet()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_err, train_acc, val_err, val_acc  = \
    train(model, train_set, test_set, epochs, batch_size, optimizer)

In [92]:
train_acc

[tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0')]